Essential libraries

In [13]:
import os
import sqlite3
from datetime import date
import pandas as pd
import pyrsm as rsm
import numpy as np

Reading Pickle File

In [14]:
bbb = pd.read_pickle("https://www.dropbox.com/s/5o6w21ty5p9526h/bbb.pkl?dl=1")
bbb.head(5)

,acctnum,gender,state,zip,zip3,first,last,book,nonbook,total,purch,child,youth,cook,do_it,reference,art,geog,buyer,training
0,10001,M,NY,10605,106,49,29,109,248,357,10,3,2,2,0,1,0,2,no,1
1,10002,M,NY,10960,109,39,27,35,103,138,3,0,1,0,1,0,0,1,no,1
2,10003,F,PA,19146,191,19,15,25,147,172,2,0,0,2,0,0,0,0,no,0
3,10004,F,NJ,07016,070,7,7,15,257,272,1,0,0,0,0,1,0,0,no,0
4,10005,F,NY,10804,108,15,15,15,134,149,1,0,0,1,0,0,0,0,no,1


In [15]:
demo = pd.read_csv('data/bbb_demographics.tsv',sep='\t')
demo.columns
bbb.columns
demo["acctnum"] = demo["acctnum"].astype(int)
demo['acctnum']

0        10001
1        10002
2        10003
3        10004
4        10005
         ...  
49995    59996
49996    59997
49997    59998
49998    59999
49999    60000
Name: acctnum, Length: 50000, dtype: int64

In [16]:
nonbook = pd.read_excel('data/bbb_nonbook.xlsx')
nonbook["acctnum"] = nonbook["acctnum"].astype(np.int32)
nonbook

,acctnum,nonbook
0,10001,248
1,10002,103
2,10003,147
3,10004,257
4,10005,134
...,...,...
49995,59996,12
49996,59997,294
49997,59998,178
49998,59999,246


Loading from SQL Lite


In [17]:
def db_list_tables(con):
    """Return all table names"""
    cursor = con.cursor()
    cursor.execute("select name from sqlite_master where type='table';")
    return [x[0] for x in cursor.fetchall()]


def db_list_fields(con, tabel):
    """Return all column names for a specified table"""
    cursor = con.cursor()
    cursor.execute(f"select * from {tabel} limit 1;")
    return [name[0] for name in cursor.description]

In [18]:
con = sqlite3.connect("data/bbb.sqlite")
buyer = pd.read_sql_query("SELECT * from buyer", con)
purchase = pd.read_sql_query("SELECT * from purchase", con)
purchase["date"] = pd.to_datetime(purchase["date"], origin="1-1-1970", unit="d")


buyer["acctnum"] = buyer["acctnum"].astype(int)
buyer["buyer"] = buyer["buyer"].astype('category')
buyer["training"] = buyer["training"].astype(np.int32)

Creating zip3 from zip

In [19]:
demo["zip"] = demo["zip"].apply(lambda x: '{0:0>5}'.format(x))
demo["zip3"] = demo["zip"].str[:3]
demo[["gender","state"]] = demo[["gender","state"]].astype('category')
demo

,acctnum,gender,state,zip,zip3
0,10001,M,NY,10605,106
1,10002,M,NY,10960,109
2,10003,F,PA,19146,191
3,10004,F,NJ,07016,070
4,10005,F,NY,10804,108
...,...,...,...,...,...
49995,59996,F,NY,11967,119
49996,59997,F,NJ,08882,088
49997,59998,M,NJ,07410,074
49998,59999,M,NJ,07090,070


Calculating diff_months

In [20]:
start_date = date(2022, 3, 8)

def diff_months(date1, date2):
    """
    This function calculates the difference in months between
    "today" (date1) and the first (last) date on which a customer
    purchased a product (date2)
    """
    if isinstance(date1, date) == False or (
        hasattr(date1, "__length__") and len(date1) > 1
    ):
        raise TypeError(f"date1 should be of type date have length 1")

    if hasattr(date2, "dtype") == False or hasattr(date2, "dt") == False:
        raise TypeError(f"date2 should be a pandas series date")

    y = date1.year - date2.dt.year
    m = date1.month - date2.dt.month
    return y * 12 + m

diff_months(start_date, purchase["date"])

0         49
1         49
2         47
3         43
4         40
          ..
194506    29
194507    29
194508    12
194509     2
194510     1
Name: date, Length: 194511, dtype: int64

Grouping and dtype fixes

In [21]:
freq = purchase.groupby("acctnum")["purchase"].value_counts()
child = pd.DataFrame(freq.loc[:, 'child']).reset_index().rename(columns = {"purchase":"child"})
child["acctnum"] = child["acctnum"].astype(int)

cook = pd.DataFrame(freq.loc[:, 'cook']).reset_index().rename(columns = {"purchase":"cook"})
cook["acctnum"] = cook["acctnum"].astype(int)

youth = pd.DataFrame(freq.loc[:, 'youth']).reset_index().rename(columns = {"purchase":"youth"})
youth["acctnum"] = youth["acctnum"].astype(int)

do_it = pd.DataFrame(freq.loc[:, 'do_it']).reset_index().rename(columns = {"purchase":"do_it"})
do_it["acctnum"] = do_it["acctnum"].astype(int)

reference = pd.DataFrame(freq.loc[:, 'reference']).reset_index().rename(columns = {"purchase":"reference"})
reference["acctnum"] = reference["acctnum"].astype(int)

art = pd.DataFrame(freq.loc[:, 'art']).reset_index().rename(columns = {"purchase":"art"})
art["acctnum"] = art["acctnum"].astype(int)

geog = pd.DataFrame(freq.loc[:, 'geog']).reset_index().rename(columns = {"purchase":"geog"})
geog["acctnum"] = geog["acctnum"].astype(int)


_purch = purchase.groupby("acctnum")["acctnum"].value_counts()
purch = pd.DataFrame(_purch.values, demo["acctnum"]).rename(columns = {0:"purch"}).reset_index()
purch["acctnum"] = purch["acctnum"].astype(int)

book = purchase.groupby("acctnum")["price"].agg("sum").reset_index().rename(columns = {"price":"book"})
book["acctnum"] = book["acctnum"].astype(int)
total = book["book"] + nonbook["nonbook"]

purch.insert(1, 'total', total.astype(np.int32))

nonbook["nonbook"] = nonbook["nonbook"].astype(np.int32)


first = pd.DataFrame(diff_months(start_date, purchase.groupby("acctnum")["date"].agg("first"))).reset_index().rename(columns = {"date":"first"})
first["acctnum"] = first["acctnum"].astype(np.int32)
first["first"] = first["first"].astype(np.int32)

last = pd.DataFrame(diff_months(start_date, purchase.groupby("acctnum")["date"].agg("last"))).reset_index().rename(columns = {"date":"last"})
last["acctnum"] = last["acctnum"].astype(np.int32)
last["last"] = last["last"].astype(np.int32)
purch

,acctnum,total,purch
0,10001,357,10
1,10002,138,3
2,10003,172,2
3,10004,272,1
4,10005,149,1
...,...,...,...
49995,59996,27,1
49996,59997,373,7
49997,59998,193,1
49998,59999,344,8


Merging

In [22]:
bbb_rec = demo.merge(first, how="outer", on='acctnum').merge(last, how="outer", on='acctnum').merge(book, how="outer", on='acctnum').merge(nonbook, how="outer", on='acctnum').merge(purch, how="outer", on='acctnum').merge(child, how="outer", on='acctnum').merge(youth, how="outer", on='acctnum').merge(cook, how="outer", on='acctnum').merge(do_it, how="outer", on='acctnum').merge(reference, how="outer", on='acctnum').merge(art, how="outer", on='acctnum').merge(geog, how="outer", on='acctnum').merge(buyer, how="outer", on='acctnum')
bbb_rec = bbb_rec.fillna(0)
bbb_rec[["purch","child","cook","youth","do_it","reference","art","geog","book"]] = bbb_rec[["purch","child","cook","youth","do_it","reference","art","geog","book"]].astype(np.int32)
bbb_rec["acctnum"] = bbb_rec["acctnum"].astype(str)

Comparison Test


In [23]:
def df_compare(df1_name, df2_name):
    df1 = globals()[df1_name]
    df2 = globals()[df2_name]
    return pd.DataFrame(
        {
            f"{df1_name} names": df1.columns,
            f"{df2_name} names": df2.columns,
            f"{df1_name} dtypes": df1.dtypes.astype(str).values,
            f"{df2_name} dtypes": df2.dtypes.astype(str),
            f"{df1_name} types": [type(c).__name__ for c in df1.iloc[0, :]],
            f"{df2_name} types": [type(c).__name__ for c in df2.iloc[0, :]],
            "names equal": df1.columns == df2.columns,
            "dtypes equal": df1.dtypes.astype(str).values
            == df2.dtypes.astype(str).values,
            "types equal": [type(c) for c in df1.iloc[0, :]]
            == [type(c) for c in df2.iloc[0, :]],
            "values equal": [all(df1[c] == df2[c]) for c in df1.columns],
        }
    )


df_compare("bbb", "bbb_rec")

,bbb names,bbb_rec names,bbb dtypes,bbb_rec dtypes,bbb types,bbb_rec types,names equal,dtypes equal,types equal,values equal
acctnum,acctnum,acctnum,object,object,str,str,True,True,True,True
gender,gender,gender,category,category,str,str,True,True,True,True
state,state,state,category,category,str,str,True,True,True,True
zip,zip,zip,object,object,str,str,True,True,True,True
zip3,zip3,zip3,object,object,str,str,True,True,True,True
first,first,first,int32,int32,int32,int32,True,True,True,True
last,last,last,int32,int32,int32,int32,True,True,True,True
book,book,book,int32,int32,int32,int32,True,True,True,True
nonbook,nonbook,nonbook,int32,int32,int32,int32,True,True,True,True
total,total,total,int32,int32,int32,int32,True,True,True,True
